## Primero

In [1]:
import numpy as np

class Agenda:
    def __init__(self, idCurso, idClase, idDocente, idSalon):
        # Inicializa una instancia de Agenda con los identificadores del curso, la clase y el docente
        self.idCurso = idCurso
        self.idClase = idClase #
        self.idDocente = idDocente
        self.idSalon =  idSalon
        # Inicializa las variables de horario con valores predeterminados
        #self.idSalon = 0
        self.diaSemana = 0
        self.horario = 0

    def Inicializador_aleatorio(self, salonRango):
        # Inicializa aleatoriamente las variables de horario dentro de ciertos rangos
        self.idSalon = np.random.choice([101, 102, 103, 104, 105, 106, 107, 108, 109, 110], 1)[0]  # Genera un numero aleatorio entre 101 y 110 (representando salones)
        self.diaSemana = np.random.randint(1, 6, 1)[0]  # Genera un numero aleatorio entre 1 y 5 (representando dias de la semana)
        self.horario = np.random.randint(1, 6, 1)[0]  # Genera un numero aleatorio entre 1 y 5 (representando horarios)

def CostoHorario(poblacion, elite):
    # Calcula el costo (conflicto) de horario para cada horario en la poblacion
    conflictos = []
    n = len(poblacion[0])  # Obtiene el numero de horarios en la poblacion (asume que todos tienen la misma cantidad)

    for p in poblacion:
        conflicto = 0  # Inicializa el contador de conflictos para este horario

        # Compara cada par de horarios para verificar si hay conflictos
        for i in range(0, n - 1):
            for j in range(i + 1, n):
                # Compara el id del salon, el dia de la semana y el horario
                if p[i].idSalon == p[j].idSalon and p[i].diaSemana == p[j].diaSemana and p[i].horario == p[j].horario:
                    conflicto += 1

                # Compara el id de la clase, el dia de la semana y el horario
                if p[i].idClase == p[j].idClase and p[i].diaSemana == p[j].diaSemana and p[i].horario == p[j].horario:
                    conflicto += 1

                # Compara el id del docente, el dia de la semana y el horario
                if p[i].idDocente == p[j].idDocente and p[i].diaSemana == p[j].diaSemana and p[i].horario == p[j].horario:
                    conflicto += 1

                # Compara el id de la clase, el id del curso y el dia de la semana
                if p[i].idClase == p[j].idClase and p[i].idCurso == p[j].idCurso and p[i].diaSemana == p[j].diaSemana:
                    conflicto += 1

        conflictos.append(conflicto)  # Agrega el numero de conflictos para este horario a la lista

    # Ordena los horarios en funcion de sus conflictos en orden ascendente
    indice = np.array(conflictos).argsort()

    # Devuelve los indices de los horarios elite (los menos conflictivos) y el numero de conflictos del mejor horario
    return indice[: elite], conflictos[indice[0]]


In [2]:
import copy
import numpy as np

#from Horario import CostoHorario

class OptimizadorGenetico:
    def __init__(self, tam_poblacion=30, prob_mutacion=0.3, elite=5, max_iteraciones=100):
        # Inicializa los parametros del algoritmo genetico
        self.tam_poblacion = tam_poblacion  # Tamaño de la poblacion de horarios candidatos
        self.prob_mutacion = prob_mutacion  # Probabilidad de aplicar mutacion a un horario
        self.elite = elite  # Numero de horarios elite que se mantendran sin cambios en cada generacion
        self.max_iteraciones = max_iteraciones  # Numero maximo de iteraciones del algoritmo genetico

    def Iniciar_poblacion(self, horarios, salonRango):
        # Crea una poblacion inicial de horarios candidatos de manera aleatoria

        self.poblacion = []

        for i in range(self.tam_poblacion):
            entidad = []
            #print('Generando horario aleatorio: {}'.format(i + 1)) # Imprime el numero de horario aleatorio generado
            # Genera horarios aleatorios y los agrega a la entidad
            for s in horarios:
                s.Inicializador_aleatorio(salonRango)
                entidad.append(copy.deepcopy(s))

            self.poblacion.append(entidad)

    def Mutar(self, poblacionElite, salonRango, i=0):
        # Aplica una mutacion en uno de los horarios de la población elite

        e = np.random.randint(0, self.elite, 1)[0]  # Elige un horario elite al azar
        pos = np.random.randint(0, 2, 1)[0]  # Decide que aspecto del horario se va a mutar (0, 1 o 2)

        ep = copy.deepcopy(poblacionElite[e])  # Copia el horario elite seleccionado.

        for p in ep:
            pos = np.random.randint(0, 3, 1)[0]  # Decide que caracteristica especifica del horario se va a mutar
            operation = np.random.rand()  # Genera un numero aleatorio para decidir si se suma o resta

            if pos == 0:
                p.idSalon = self.AgregarResta(p.idSalon, operation, salonRango)  # Mutacion en el id del salon
            if pos == 1:
                p.diaSemana = self.AgregarResta(p.diaSemana, operation, 5)  # Mutacion en el dia de la semana
            if pos == 2:
                p.horario = self.AgregarResta(p.horario, operation, 5)  # Mutacion en el horario

        #print('Mutacion en el horario: {}'.format(e + 1))  # Imprime el numero de horario mutado
        return ep

    def AgregarResta(self, valor, op, valorRango):
        # Realiza una operacion de suma o resta en funcion de la probabilidad op

        if op > 0.5:
            if valor < valorRango:
                valor += 1
            else:
                valor -= 1
        else:
            if valor - 1 > 0:
                valor -= 1
            else:
                valor += 1

        return valor

    def Cruzar(self, poblacionElite):
        # Realiza un cruce (intercambio) entre dos horarios de la población olite

        e1 = np.random.randint(0, self.elite, 1)[0]  # Elige un primer horario elite al azar
        e2 = np.random.randint(0, self.elite, 1)[0]  # Elige un segundo horario elite al azar

        pos = np.random.randint(0, 2, 1)[0]  # Decide que aspecto del horario se va a cruzar (0 o 1)

        ep1 = copy.deepcopy(poblacionElite[e1])  # Copia el primer horario elite seleccionado
        ep2 = poblacionElite[e2]  # Toma el segundo horario elite.

        for p1, p2 in zip(ep1, ep2):
            if pos == 0:
                p1.diaSemana = p2.diaSemana  # Cruce en el dia de la semana
                p1.horario = p2.horario  # Cruce en el horario
            if pos == 1:
                p1.idSalon = p2.idSalon  # Cruce en el id del salon

        #print('Cruce entre los horarios: {} y {}'.format(e1 + 1, e2 + 1))  # Imprime los numeros de horarios cruzados
        return ep1

    def Evolucion(self, horarios, salonRango):
        # Realiza la evolucion de la poblacion a lo largo de varias iteraciones

        mejorPunto = 0
        mejorHorario = None

        self.Iniciar_poblacion(horarios, salonRango)  # Inicializa la poblacion aleatoriamente

        for i in range(self.max_iteraciones):
            indiceElite, mejorPunto = CostoHorario(self.poblacion, self.elite)  # Evalua y selecciona a la elite
            #print('---------------------------------------------------------------')
            #print('Iteracion: {} | conflicto: {} | Tamaño de la población elite: {}'.format(i + 1, mejorPunto, len(indiceElite)))
            #print('---------------------------------------------------------------')

            if mejorPunto == 0:
                mejorHorario = self.poblacion[indiceElite[0]]
                break

            nuevaPoblacion = [self.poblacion[indice] for indice in indiceElite]

            while len(nuevaPoblacion) < self.tam_poblacion:
                if np.random.rand() < self.prob_mutacion:
                    nuevaP = self.Mutar(nuevaPoblacion, salonRango)  # Aplica mutacion con cierta probabilidad
                else:
                    nuevaP = self.Cruzar(nuevaPoblacion)  # Realiza cruce entre horarios elite

                nuevaPoblacion.append(nuevaP)

            self.poblacion = nuevaPoblacion  # Actualiza la poblacion

        return mejorHorario


In [3]:
import prettytable

#from Horario import Agenda
#from alg_genetico import OptimizadorGenetico

def vis(VHorario):
    # función para visualizar el horario de clases de forma tabular
    etiqueta_colum = ['Hora', 'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']  # Etiquetas de las columnas (horas y días)

    # Matriz de valores para la tabla [horas, lunes, martes, miércoles, jueves, viernes
    tabla_valores = [
        ["7 am", "", "", "", "", ""],
        ["8 am", "", "", "", "", ""],
        ["9 am", "", "", "", "", ""],
        ["10 am", "", "", "", "", ""],
        ["11 am", "", "", "", "", ""],
        ["12 pm", "", "", "", "", ""],
    ]

    # Crea una tabla con las etiquetas y bordes
    # etiqueta_colum: etiquetas de las columnas
    # hrules: estilo de los bordes, prettytable.ALL: todos los bordes
    tabla = prettytable.PrettyTable(etiqueta_colum, hrules=prettytable.ALL)

    for s in VHorario:
        # Se obtienen los valores de la clase
        # Se crea un string con la información de la clase
        # Se agrega el string a la matriz en la posición correspondiente
        diaSemana = s.diaSemana
        horario = s.horario
        texto = 'Curso: {} \n Clase: {} \n Salon: {} \n Profesor: {}'.format(s.idCurso, s.idClase, s.idSalon, s.idDocente)
        tabla_valores[diaSemana - 1][horario] = texto  # Agrega la información del horario en la matriz

        # Si
        if horario == 5:
            tabla_valores[5][diaSemana] = texto  # Agrega la información de la clase en la fila de "12 pm"

    for row in tabla_valores:
        tabla.add_row(row)  # Agrega cada fila de la matriz a la tabla

    print(tabla)  # Imprime la tabla visual del horario

if __name__ == '__main__':
    horarios = []

    # Se crean instancias de Agenda con diferentes combinaciones de identificadores de curso, clase y docente
    horarios.append(Agenda("Algebra", "IF061AIN", "Docente 1", 101))
    horarios.append(Agenda("Algebra", "IF061AIN", "Docente 1", 101))
    horarios.append(Agenda("A.D.A.", "IF061AIN", "Docente 2", 102))
    horarios.append(Agenda("A.D.A.", "IF061AIN", "Docente 2", 102))
    horarios.append(Agenda("Avanzados", "IF061AIN", "Docente 3", 103))
    horarios.append(Agenda("Avanzados", "IF061AIN", "Docente 3", 103))
    horarios.append(Agenda("M. Discreta", "IF061AIN", "Docente 6", 106))
    horarios.append(Agenda("M. Discreta", "IF061AIN", "Docente 6", 106))

    horarios.append(Agenda("A.D.A.", "IF484AIN", "Docente 2", 102))
    horarios.append(Agenda("A.D.A.", "IF484AIN", "Docente 2", 102))
    horarios.append(Agenda("Programacion 1", "IF484AIN", "Docente 4", 104))
    horarios.append(Agenda("Programacion 1", "IF484AIN", "Docente 4", 104))
    horarios.append(Agenda("M. Discreta", "IF484AIN", "Docente 6", 106))
    horarios.append(Agenda("M. Discreta", "IF484AIN", "Docente 6", 106))

    horarios.append(Agenda("Avanzados", "IF554AIN", "Docente 3", 103))
    horarios.append(Agenda("Avanzados", "IF554AIN", "Docente 3", 103))
    horarios.append(Agenda("Programacion 1", "IF554AIN", "Docente 4", 104))
    horarios.append(Agenda("Programacion 1", "IF554AIN", "Docente 4", 104))
    horarios.append(Agenda("BioInformatica", "IF554AIN", "Docente 5", 105))
    horarios.append(Agenda("BioInformatica", "IF554AIN", "Docente 5", 105))
    horarios.append(Agenda("M. Discreta", "IF554AIN", "Docente 6", 106))
    horarios.append(Agenda("M. Discreta", "IF554AIN", "Docente 6", 106))

    ga = OptimizadorGenetico(tam_poblacion=50, elite=10, max_iteraciones=500)  # Se crea una instancia del Optimizador Genetico.

    resultado = ga.Evolucion(horarios, 3)  # Se ejecuta el algoritmo genetico para encontrar un horario optimo.

    horario_visualizado = []
    for r in resultado:
        if r.idClase == "IF554AIN":
            horario_visualizado.append(r)  # Se seleccionan los horarios específicos de la clase 1203.

    vis(horario_visualizado)  # Se visualiza el horario resultante.


+-------+------------------------+------------------------+------------------------+--------+----------------------+
|  Hora |         Lunes          |         Martes         |       Miércoles        | Jueves |       Viernes        |
+-------+------------------------+------------------------+------------------------+--------+----------------------+
|  7 am |   Curso: Avanzados     |                        |  Curso: M. Discreta    |        |                      |
|       |    Clase: IF554AIN     |                        |    Clase: IF554AIN     |        |                      |
|       |       Salon: 110       |                        |       Salon: 108       |        |                      |
|       |   Profesor: Docente 3  |                        |   Profesor: Docente 6  |        |                      |
+-------+------------------------+------------------------+------------------------+--------+----------------------+
|  8 am |  Curso: M. Discreta    |                        |     